# Getting Started

This document provides an overview of some of DSS-Python's features, especially for new users.

There are other documents for specific features, and a lot of other features still need documentation for new users.

**Notebook requirements**

This is a Jupyter notebook. Running it as-is will try to download the required files.

You can open and then run this notebook on Google Colab for a quick overview if you don't want to set up a local environment: **[Open in Colab](https://colab.research.google.com/github/dss-extensions/dss_python/blob/master/docs/examples/GettingStarted.ipynb)**.

In [ ]:
# When running via Colab, install the package first
import os, subprocess
if os.getenv("COLAB_RELEASE_TAG"):
    print(subprocess.check_output('pip install dss-python[all]', shell=True).decode())

In [ ]:
# Download the sample circuits and test cases if they do not exist already
from dss.examples import download_repo_snapshot
IEEE13_PATH = download_repo_snapshot('.', repo_name='electricdss-tst', use_version=False) / 'Version8/Distrib/IEEETestCases/13Bus/IEEE13Nodeckt.dss'
assert IEEE13_PATH.exists()

If you are reading this notebook online and would like to run in your local Python installation, you can download both the DSS-Python repository (which contains this notebook) and the sample circuits with `download_examples`.

On Windows, run on a command prompt in your Python environment to install all optional dependencies:

```batch
pip install dss-python[all]
python -c "from dss.examples import download_examples; download_examples(r'c:\temp')"
cd c:\temp\dss_python\docs\examples
jupyter lab
```


## Introduction

`DSS-Python` is an effort in the [DSS-Extensions](https://dss-extensions.org/) project. As such, it doesn't require EPRI's OpenDSS to be installed. It provides its own customized engine, which in turn enables us to run the DSS engine on Windows, Linux and macOS (including newer Apple ARM processors, a.k.a. "Apple Silicon").

For a comparison of the general Python-level API, including a list of our extra functions, please check [DSS-Extensions — OpenDSS: Overview of Python APIs](https://github.com/dss-extensions/dss-extensions/blob/main/docs/python_apis.md). That documents introduces and compares DSS-Python, OpenDSSDirect.py, and the official COM implementation.

To use `DSS-Python`, after installation, open a Python interpreter and type the following command:

In [ ]:
from dss import dss

The package exposes the lower level API functions from AltDSS/DSS C-API mimicking the organization and behavior of the official COM implementation, as used in Python. This allows an easier migration, or even toggling which interface is used if the user avoids using API Extensions (which are marked as such in the documentation).

To keep the code compatible with the COM implementation, users can feed commands through the text interface:

In [ ]:
dss.Text.Command = f'Redirect "{IEEE13_PATH}"'

Alternatively, if there is no need to maintain this compatibility, there is a shortcut function that allows both single commands and multiple commands (passed through multiline strings):

In [ ]:
dss(f'Redirect "{IEEE13_PATH}"')

After a DSS circuit is loaded, the interaction can be done as with the official COM module:

In [ ]:
dssCircuit = dss.ActiveCircuit
dssCircuit.AllBusNames

In [ ]:
dssLoads = dssCircuit.Loads
idx = dssLoads.First

while idx:
    print(dssLoads.Name)
    idx = dssLoads.Next

print(f'We have {dssLoads.Count} loads in this circuit')

You can also use more Pythonic iteration. General OpenDSS caveats related to the classic OpenDSS API, which only allows one active object for each class, still apply.

In [ ]:
for load in dssLoads:
    print(load.Name)

print(f'We have {len(dssLoads)} loads in this circuit')

## Use the enums

Instead of using magic numbers like in `dss.ActiveCircuit.Solution.Mode = 1`, use the enums. Import the whole `enums` module, or separate enums you use. You can import the enums from the `dss` for short:

In [ ]:
from dss.enums import SolveModes
dss.ActiveCircuit.Solution.Mode = SolveModes.Daily

from dss import SolveModes
dss.ActiveCircuit.Solution.Mode = SolveModes.Daily

Using magic numbers is bad for multiple reasons:

- Harder to read for other users. Each user has to each memorize every value or constantly check the reference.
- The values can actually change throughout the releases. It already happened in the past in OpenDSS and some bugs persistent for about 15 years!
- Using the provided enum classes ensure, in most cases, that you are passing a valid value. Currently, DSS-Python or most Python APIs for OpenDSS do not enforce or check the values, so using the correct enum can reduce the chance of accidents.

See the list of enumerations, including important comments, in [dss.enums](https://dss-extensions.org/dss_python/enumerations.html) page.

## Migrating from the COM implementation

A lot of users start using OpenDSS using the COM implementation, which is the official OpenDSS version, limited to Windows as of February 2024.

That is fine and could even be the recommended path. Later, users can either migrate or toggle which version is used. Using DSS-Python should make that easier, since it is API-compatible with the COM implementation (but does not use COM at all). Users could include code that use OpenDSSDirect.py or AltDSS-Python to complement aspect that the COM-like approach is not sufficient.

For example, suppose you had the following code running in the official OpenDSS via COM, using `win32com` or `comtypes` (Windows only, etc.):

```python
# Load OpenDSS
import win32com.client
dss = win32com.client.Dispatch('OpenDSSengine.DSS')

# ...or for win32com with ensured early bindings:
# import win32com.client
# dss = win32com.client.gencache.EnsureDispatch('OpenDSSengine.DSS')

# ...or:
# import comtypes.client
# dss = comtypes.client.CreateObject('OpenDSSengine.DSS')

# Run a DSS script to load a circuit
# (NOTE: typically you would use either the full path here since the official OpenDSS implementation changes the current working directory of the process)
dss.Text.Command = f'Redirect "{IEEE13_PATH}"'

# Select a load and update its kW
dss.ActiveCircuit.Loads.Name = "675c"
dss.ActiveCircuit.Loads.kW = 320

# Solve
dss.ActiveCircuit.Solution.Solve()

# Get the voltage magnitudes
voltages = dss.ActiveCircuit.AllBusVmag
```

You could use DSS-Python to keep the code with a few changes.

In [ ]:
# Load DSS-Python
from dss import dss

# Run a DSS script to load a circuit
# (DSS-Extensions do not change the CWD when importing, relative paths are fine)
dss.Text.Command = f'Redirect "{IEEE13_PATH}"'

# Select a load and update its kW
dss.ActiveCircuit.Loads.Name = "675c"
dss.ActiveCircuit.Loads.kW = 320

# Solve
dss.ActiveCircuit.Solution.Solve()

# Get the voltage magnitudes
voltages = dss.ActiveCircuit.AllBusVmag

Only the first lines of the original code were changed.

### Capitalization

Sometimes, users are not aware of early bindings and the attribute lookup on `win32com` becomes case-insensitive.

To make it easier to migrate in those situations, DSS-Python includes a special setup that makes if case-insensitive, with the extra option to allow it to warn when the name capitalization does not match the expected, [set_case_insensitive_attributes](https://dss-extensions.org/dss_python/apidocs/dss/dss._cffi_api_util.html#dss._cffi_api_util.set_case_insensitive_attributes). 

Since `set_case_insensitive_attributes` has a small performance overhead, it is important to not overuse it. The warnings allow the user to track and fix capitalization issues, and finally remove the use of `set_case_insensitive_attributes` when no more warnings remain.

In [ ]:
from dss import set_case_insensitive_attributes

set_case_insensitive_attributes(use=True, warn=True)

print(dss.activecircuit.Loads.kW) # this produces a warning
print(dss.ActiveCircuit.Loads.kvar) # this one works fine since the capitalization is correct

### Bringing some Python back to the COM version

Sometimes keeping compatibility with both implementations (DSS-Extensions and EPRI) is useful or even required. Users can be spoiled by some simple quality of life improvements from DSS-Python, like the iterators.

To make things easier, DSS-Python provides a function, `patch_dss_com`, to patch the COM classes with some extras. This function does not change any aspect of the official OpenDSS engine, it just provides some Python functionality. It will, of course, require DSS-Python to be installed, but this will use the COM DLL and can be a quick way to try a script in both versions:

```python
import comtypes, dss
dss_com = dss.patch_dss_com(comtypes.client.CreateObject("OpenDSSEngine.DSS"))
print(dss_com.Version)

# ...compile a circuit, etc.

for l in dss_com.ActiveCircuit.Loads:
    print(l.Name, l.kva)

for b in dss_com.ActiveCircuit.ActiveBus:
    print(b.Name, b.x, b.y)
```

This works with both `comtypes` and `win32com`.

There is a related effort to provide a lower level implementation to be shared across all DSS-Extensions. In the mean time, `patch_dss_com` can help.

## Better numeric types

For backwards compatibility, including with the COM implementation, DSS-Python uses simple numeric types and arrays for results, float, int and 1d arrays.

There is an toggle for `AdvancedTypes` that enable complex numbers and matrices:

In [ ]:
# Setting numpy to avoid wrapping the text output
import numpy as np
np.set_printoptions(linewidth=200)


In [ ]:
dss.AdvancedTypes = True

dss.ActiveCircuit.AllBusVolts

In [ ]:
dss.ActiveCircuit.Lines.idx = 6
dss.ActiveCircuit.ActiveCktElement.Yprim

There is a lot of code that doesn't expect this modern output, so it can be toggled as required:

In [ ]:
dss.AdvancedTypes = False
dss.ActiveCircuit.ActiveCktElement.Yprim

## Multiple engines in the same process

For a full example, visit [Multiple DSS engines, multithreading vs. multiprocessing](https://dss-extensions.org/dss_python/examples/Multithreading.html).

A short example without multithreading:

In [ ]:
from dss import dss
import numpy as np
dss.AllowChangeDir = False

NUM_ENGINES = 5
load_mults = np.linspace(0.8, 1.2, NUM_ENGINES)
engines = [dss.NewContext() for _ in load_mults]

for load_mult, engine in zip(load_mults, engines):
    engine.Text.Command = f'Redirect "{IEEE13_PATH}"'
    engine.ActiveCircuit.Solution.LoadMult = load_mult
    engine.ActiveCircuit.Solution.Solve()

for engine in engines:
    lm = engine.ActiveCircuit.Solution.LoadMult
    losses = engine.ActiveCircuit.Losses[0]
    print(f'{lm:5.3} {losses:10.1f}')


## Plotting and notebook integration

For more examples on how to use the plot commands from OpenDSS, including an extensive gallery, see [Integrated plotting in Python](https://dss-extensions.org/dss_python/examples/Plotting.html).

A short example:

In [ ]:
from dss import dss, plot
plot.enable()

dss.Text.Command = f'Redirect "{IEEE13_PATH}"'
dss.Text.Command = 'plot circuit'

This enables the cell magic function and other enhancements. There is still a lot of room for improvement here.

In [ ]:
%%dss
visualize voltages element=transformer.sub 

## JSON exports and imports

Another on-going effort is support for JSON export and import: https://dss-extensions.org/dss_python/examples/JSON.html

Some highlights:

In [ ]:
import json
from dss import dss
import numpy as np

dss(f'Redirect "{IEEE13_PATH}"')
dss.ActiveCircuit.Loads.First;
json.loads(dss.ActiveCircuit.ActiveDSSElement.ToJSON())

In [ ]:
import pandas as pd
dss.ActiveCircuit.SetActiveClass('load')
df = pd.read_json(dss.ActiveCircuit.ActiveClass.ToJSON(), dtype_backend='pyarrow')
df.dtypes

In [ ]:
df

## Integration with OpenDSSDirect.py and AltDSS-Python

Assuming they are installed, there are two handy functions to map the DSS context to the other packages:

In [ ]:
from dss import dss

dss(f'Redirect "{IEEE13_PATH}"')

In [ ]:
odd = dss.to_opendssdirect()
print(odd.Version())
print()
alt = dss.to_altdss()
print(alt.Version())

In [ ]:
load = alt.Load[0]
load, load.Powers(), load.to_json()